<a href="https://colab.research.google.com/github/pszachew/Music-Genre-Classification/blob/master/training_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
X = pd.read_csv('/content/gdrive/MyDrive/csv_data/embedded.csv',header=None)

In [14]:
y = pd.read_csv('/content/gdrive/MyDrive/csv_data/labels.csv', header=None)

In [18]:
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=10)

## **Logistic Regression**

In [17]:
from sklearn.linear_model import LogisticRegression

In [20]:
log_reg_mod = LogisticRegression(max_iter=1000)

In [21]:
log_reg_mod.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
joblib.dump(log_reg_mod, 'log_reg.sav')

['log_reg.sav']

In [28]:
y_pre_lg = log_reg_mod.predict(X_test)

In [29]:
print(classification_report(y_test,y_pre_lg))

              precision    recall  f1-score   support

       blues       0.85      0.94      0.89        35
   classical       0.92      1.00      0.96        33
     country       0.74      0.74      0.74        27
       disco       0.86      0.74      0.79        34
      hiphop       0.74      0.96      0.84        24
        jazz       1.00      0.91      0.96        35
       metal       0.97      1.00      0.98        31
         pop       0.79      0.88      0.84        26
      reggae       0.92      0.77      0.84        30
        rock       0.70      0.56      0.62        25

    accuracy                           0.86       300
   macro avg       0.85      0.85      0.85       300
weighted avg       0.86      0.86      0.85       300



In [38]:
print(confusion_matrix(y_test, y_pre_lg))

[[33  1  0  0  0  0  0  0  0  1]
 [ 0 33  0  0  0  0  0  0  0  0]
 [ 3  0 20  2  0  0  0  1  0  1]
 [ 0  0  2 25  2  0  0  2  0  3]
 [ 0  0  0  0 23  0  0  0  1  0]
 [ 1  2  0  0  0 32  0  0  0  0]
 [ 0  0  0  0  0  0 31  0  0  0]
 [ 0  0  0  0  2  0  0 23  0  1]
 [ 0  0  2  1  2  0  0  2 23  0]
 [ 2  0  3  1  2  0  1  1  1 14]]
